In [1]:
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from tqdm import tqdm
from fuzzywuzzy import fuzz
import random
import re

In [3]:
#Leemos nuestro csv con los datos totales de peliculas y cortos extaidos de la api MovieDatabase 
lista_api=pd.read_csv('sin_genero.csv')
lista_api.tail()

,Tipo,Nombre,Año,Mes,Id
1702,Short,One Last Thing,2020,No Info,tt10360616
1703,Movie,Im toten Winkel,2020,3.0,tt10366464
1704,Movie,Paper Tiger,2020,8.0,tt10365730
1705,Movie,Rebirth,2020,2.0,tt10369332
1706,Short,Cherry Swamp Road,2020,5.0,tt10369880


In [25]:
#Hacemos un random para la muestra

lista_tuplas = [tuple(x) for x in lista_api.values]
lista_tuplas= random.sample(lista_tuplas, 1706)


In [26]:
lista_tuplas= lista_tuplas[1:3]
print(lista_tuplas)

[('Movie', 'They Who Surround Us', 2020, '8.0', 'tt10042622'), ('Movie', 'El verano que vivimos', 2020, '12.0', 'tt10042590')]


In [27]:
def extraccion_caracteristicas_peliculas(url): 
    
    driver = webdriver.Chrome()
    driver.get(url)
    sleep(2)
    driver.maximize_window()
 # Aceptamos las cookies
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    sleep(2)
    

    lista_tupla_imdb=[]

    for tupla in tqdm(lista_tuplas):

        lista_imdb =[]
        
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)

            
        id_pelicula=tupla[-1]    
        lista_imdb.append(id_pelicula)
            
        sleep(1)
            
        nombre_pelicula=tupla[1]
        lista_imdb.append(nombre_pelicula)
            
        tipo=tupla[0]
        lista_imdb.append(tipo)
            
        sleep(1)
        try: 
                
                try: 
                    director= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[1]/div').text
                    director_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', director)   
                    lista_imdb.append(director_correcto) 
                except: 
                    director=driver.find_element('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[1]/section/div[2]/div/ul/li[1]/div').text
                    director_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', director)   
                    lista_imdb.append(director_correcto)
            
        except: 
                
            lista_imdb.append("No especificado")
            
        sleep(2)
            
        try: 
                try: 
                    guionista= driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[2]/div/ul/li[2]/div').text
                    guionista_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', guionista)
                    lista_imdb.append(guionista_correcto)
                except: 
                    guionista=driver.find_element('xpath', '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[1]/section/div[2]/div/ul/li[2]/div').text
                    guionista_correcto = re.sub(r'([a-z])([A-Z])', r'\1 \2', guionista)
                    lista_imdb.append(guionista_correcto)
                
            
        except: 
            lista_imdb.append("No especificado")
            
        sleep(2)
            
        try: 
                for i in range(2, 4):
                    duracion= driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[{i}]').text
                    if 'min' in duracion or 'h' in duracion: 
                        lista_imdb.append(duracion)
                        break
                    else:
                        pass           
        except:
            duracion =("No especificado")
            lista_imdb.append(duracion)
                                
                            
        sleep(3)
            
        try: 
            puntuacion = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text
            
            lista_imdb.append(puntuacion.replace(",", "."))
                
        except: 
            lista_imdb.append("No especificado")

        sleep(1)
            
        try:  
                
            #print("el argumento") 
            argumento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
                                                        
            lista_imdb.append(argumento)
            
        except:
            #print("sin argumento")
            lista_imdb.append("No especificado")
            
        sleep(2)
                
        lista_tupla_imdb.append(tuple(lista_imdb))

        sleep(4)
        driver.back()
    
    return lista_tupla_imdb
        

In [28]:
resultados= extraccion_caracteristicas_peliculas("https://www.imdb.com/")

100%|██████████| 2/2 [01:03<00:00, 31.88s/it]


In [29]:
caracteristicas_peliculas=pd.DataFrame(resultados, columns=["Id", "Nombre","Tipo", "Director", "Guionista", "Duracion", "Puntuacion", "Argumento"])

caracteristicas_peliculas.head()
caracteristicas_peliculas.drop_duplicates()

caracteristicas_peliculas.to_csv("caracteristicas_imdb.csv")

In [31]:
caracteristicas_peliculas

,Id,Nombre,Tipo,Director,Guionista,Duracion,Puntuacion,Argumento
0,tt10042622,They Who Surround Us,Movie,Troy Ruptash,Troy Ruptash,1h 28min,4.5,A Ukrainian farmer living in Alberta loses his...
1,tt10042590,El verano que vivimos,Movie,Carlos Sedes,Ramón Campos Javier Chacártegui Salvador S. Mo...,1h 59min,5.9,"En 1998, Isabel es una estudiante de periodism..."


In [156]:
pelis_imdb= pd.read_csv("caracterist_imdb.csv")

In [157]:
pelis_imdb

,ID,Nombre,Tipo,Director,Guionista1,Guionista2,Duración,Puntuación,Argumento
0,tt0177220,Shulie,Short,Elisabeth Subrin,No especificado,No especificado,No especificado,No especificado,No especificado
1,tt0297400,Snowblind,Movie,No especificado,No especificado,No especificado,No especificado,No especificado,No especificado
2,tt0337926,Chatô: O Rei do Brasil,Movie,Guilherme Fontes,João Emanuel Carneiro,Guilherme Fontes,1h 42min,5.9,"La historia de Assis Chateaubriand, el primer ..."
3,tt0346045,Transeúntes,Movie,Luis Aller,Luis Aller,No especificado,1h 34min,6.7,"Passers-by is a constructivist, multiple, frag..."
4,tt0420293,The Stanford Prison Experiment,Movie,Kyle Patrick Alvarez,Tim Talbott,Philip Zimbardo,2h 2min,6.8,"En 1971, veinticuatro estudiantes masculinos s..."
...,...,...,...,...,...,...,...,...,...
1702,tt10360616,One Last Thing,Short,Richie Starzec,No especificado,No especificado,No especificado,No especificado,No especificado
1703,tt10366464,Im toten Winkel,Movie,Michael Roitzsch,No especificado,No especificado,No especificado,No especificado,No especificado
1704,tt10365730,Paper Tiger,Movie,Paul Kowalski,No especificado,No especificado,No especificado,No especificado,No especificado
1705,tt10369332,Rebirth,Movie,No especificado,No especificado,No especificado,No especificado,No especificado,No especificado


In [158]:
lista_peliculas = [tuple(x) for x in pelis_imdb.values]

In [159]:
lista_peliculas


[('tt0177220',
  'Shulie',
  'Short',
  'Elisabeth Subrin',
  'No especificado',
  'No especificado',
  'No especificado',
  'No especificado',
  'No especificado'),
 ('tt0297400',
  'Snowblind',
  'Movie',
  'No especificado',
  'No especificado',
  'No especificado',
  'No especificado',
  'No especificado',
  'No especificado'),
 ('tt0337926',
  'Chatô: O Rei do Brasil',
  'Movie',
  'Guilherme Fontes',
  'João Emanuel Carneiro',
  'Guilherme Fontes',
  '1h 42min',
  '5.9',
  'La historia de Assis Chateaubriand, el primer magnate de la comunicación de Brasil. Debido a su influencia desde finales de la década de 1930 hasta principios de la de 1960, se le ha llamado El ciudadano brasileño Kane.'),
 ('tt0346045',
  'Transeúntes',
  'Movie',
  'Luis Aller',
  'Luis Aller',
  'No especificado',
  '1h 34min',
  '6.7',
  "Passers-by is a constructivist, multiple, fragmented and cosmic film... a cinematographic Big Bang resulting from a thorough conception and edition work by Director Luis 

In [160]:
lista_peliculas= lista_peliculas[298:650]
print(lista_peliculas)

[('tt10015802', 'The Big 3', 'Movie', 'Tye Banks', 'Bo Alambis', 'Ampston Hews', '46min', '9.3', "Liquor House Comedy presents the Big 3 featuring stand up comedians Brian Isley (New York's Apollo Theater) Bo Alambis and Ampston Hews. None of these three comedians are a like. They all bring a unique style of stand up comedy."), ('tt0926132', 'Road to Red', 'Movie', 'Tito da Costa', 'Larry Leahy', 'No especificado', 'No especificado', '3.7', 'A surf-skate action-thriller about five best friends who set out on a road trip that quickly turns into an epic battle for survival.'), ('tt0983946', 'Fantasy Island', 'Movie', 'Jeff Wadlow', 'Jeff Wadlow', 'Christopher Roach', '1h 49min', '4.9', 'El Sr. Roarke hace realidad los sueños de sus huéspedes. Sin embargo, cuando todo se convierte en una pesadilla, los invitados tienen que resolver el misterio de la isla para poder escapar con vida.'), ('tt10011228', 'Hunted', 'Movie', 'Vincent Paronnaud', 'Vincent Paronnaud', 'No especificado', '1h 27min

In [9]:
def diez_actores(lista_peliculas):  
    
    driver = webdriver.Chrome()

    driver.maximize_window()

    driver.get("https://www.imdb.com/")

    lista_tupla_actores = []
    
    sleep(3)
    
    # Aceptamos las cookies
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    sleep(2)
    lista_tupla_actores = []

    for tupla in tqdm(lista_peliculas):
        
        driver.find_element("css selector", "#suggestion-search").send_keys(tupla[0], Keys.ENTER)
        sleep(3)
        
        lista_actores = []
        for i in range(1, 11):
            try:
                lista_actores.append(driver.find_element("css selector", f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i}) > div.sc-bfec09a1-7.gWwKlt > a').text)
                
            except:            
                break
        lista_actores.append(tupla[1])
        lista_tupla_actores.append(tuple(lista_actores))

        driver.back()

    driver.close()   
    
    return lista_tupla_actores


In [15]:
actores_por_peli=diez_actores(lista_peliculas)

NameError: name 'lista_peliculas' is not defined

In [14]:
actores_por_peli

NameError: name 'actores_por_peli' is not defined

In [13]:
actores_por_peli4 = pd.DataFrame(actores_por_peli)


NameError: name 'actores_por_peli' is not defined

In [165]:
actores_por_peli4

,0,1,2,3,4,5,6,7,8,9,10
0,Bo Alambis,Ampston Hews,Brian Isley,Dink Kearney,The Big 3,None,None,None,None,None,None
1,Sean Gray,Chris Blasman,Renee Dorian,Michael Buffo,Amanda Wing,Matt Lasky,Mike Saad,Buddy Daniels Friedman,Holt Boggs,Matthew Prater,Road to Red
2,Michael Peña,Maggie Q,Lucy Hale,Austin Stowell,Jimmy O. Yang,Portia Doubleday,Ryan Hansen,Michael Rooker,Parisa Fitz-Henley,Mike Vogel,Fantasy Island
3,Lucie Debay,Arieh Worthalter,Ciaran O'Brien,Ryan Brodie,Simone Milsdochter,Mikael Sladden,Bruce Ellison,Gilles Vandeweerd,Leila Putcuips,Clélia Zanini,Hunted
4,Jiiva,Navdeep,Riya Suman,Varun Kamal,Sathish,R.N.R. Manohar,Fighter Karthi,Sekar P.R.,Seeru,None,None
...,...,...,...,...,...,...,...,...,...,...,...
347,Jerilyn Armstrong,Russell Becker,Adam Halferty,Jessie Lynn Her,Ryan Kattner,DeVaughn LaBon,Flora Rubenhold,James Russo,Jason Tippet,Woe,None
348,Rohain Arora,Cory Bertrand,Gregory Terlecki,Deborah Lobban,Scott McCulloch,Claudelle Tremblay,Cameron Stannard,Myrna Wolfe-Maxwell,Brian De La Franier,Paul Perreault,The Devil's Tail
349,Genesis Lynea,David O'Mahony,Shaun Cowlishaw,Dev Joshi,Louise North,Roberts Balanas,Wanjiku Gitau,Luca Keaney,Rosanne Jade Miller,Simon Morris,How Can I Forget
350,Ayushmann Khurrana,Jitendra Kumar,Gajraj Rao,Neena Gupta,Manu Rishi Chadha,Sunita Rajwar,Maanvi Gagroo,Pankhuri Awasthi,Neeraj Singh,Gopal Datt,Shubh Mangal Zyada Saavdhan


In [12]:
actores_por_peli4.to_csv('actores_por_peli.csv', index=False)

NameError: name 'actores_por_peli4' is not defined

In [167]:
actores_por_peli4= actores_por_peli4[1:3]

In [10]:
actores_sin_peliculas = [tupla[:-1] for tupla in actores_por_peli] # Pasamos a una lista de tuplas los actores y quitamos las peliculas

print(actores_por_peli)

NameError: name 'actores_por_peli' is not defined

In [172]:
actores_sin_peliculas 

[('Bo Alambis', 'Ampston Hews', 'Brian Isley', 'Dink Kearney'),
 ('Sean Gray',
  'Chris Blasman',
  'Renee Dorian',
  'Michael Buffo',
  'Amanda Wing',
  'Matt Lasky',
  'Mike Saad',
  'Buddy Daniels Friedman',
  'Holt Boggs',
  'Matthew Prater'),
 ('Michael Peña',
  'Maggie Q',
  'Lucy Hale',
  'Austin Stowell',
  'Jimmy O. Yang',
  'Portia Doubleday',
  'Ryan Hansen',
  'Michael Rooker',
  'Parisa Fitz-Henley',
  'Mike Vogel'),
 ('Lucie Debay',
  'Arieh Worthalter',
  "Ciaran O'Brien",
  'Ryan Brodie',
  'Simone Milsdochter',
  'Mikael Sladden',
  'Bruce Ellison',
  'Gilles Vandeweerd',
  'Leila Putcuips',
  'Clélia Zanini'),
 ('Jiiva',
  'Navdeep',
  'Riya Suman',
  'Varun Kamal',
  'Sathish',
  'R.N.R. Manohar',
  'Fighter Karthi',
  'Sekar P.R.'),
 ('Joshua Garcia',
  'Julia Barretto',
  'Ian Veneracion',
  'Dimples Romana',
  'Maris Racal',
  'McCoy De Leon',
  'Yves Flores',
  'Ina Raymundo',
  'Miel Espinoza',
  'Myrtle Sarrosa'),
 ('Jaden Luo',
  'Xavier Luo',
  'Ira Grossman'

In [31]:
actores_peli = pd.read_csv("actores_por_peli.csv")
lista_tupla_actores = [tuple(x) for x in actores_peli.values]
lista_tupla_actores = lista_tupla_actores[0:100]

In [34]:
def caracteristicas_actores(lista_tupla_actores):
    
    driver = webdriver.Chrome()
    driver.maximize_window()

    driver.get("https://www.imdb.com/")
    lista_detalles_actores = []

# Aceptamos las cookies
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
    sleep(2)
    
    for tupla in lista_tupla_actores:
        
        for actor in tqdm(tupla[1:]):
        
            print(actor)
            sleep(7)
            driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)
                
            sleep(5)
            
            # Click en el primer resultado de búsqueda
            try:
                #driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-872d7ac7-0.fqEQWL > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1)").click()
                driver.find_element("css selector", '#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-3c609de6-0.gDoIHY > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(4) > div.sc-ffc93fc1-2.ditJlF > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a').click()
            except:
                print("No se pudo hacer clic en el primer resultado de búsqueda.")
                continue  # Saltar al siguiente actor si no se puede hacer clic en el primer resultado

            lista_caracteristicas = []

            #Fecha nacimiento
            try:
                anio = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text.split(" ")[-1]
                    
                lista_caracteristicas.append(anio)                  
                
            except:            
                lista_caracteristicas.append("Desconocido")
            
            #Top de peliculas conocidas    
   

            try: 
                conocidas = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]').text
            except: 
                conocidas="Sin datos"
            
            #Sacamos las profesiones     
            try:   
                profesiones = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text.replace("\n", ', ')
            
            except:
                profesiones="Sin datos"
            
            try: 
                oscar= driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/a[1]').text
                if "Ganó" in oscar: 
                    lista_caracteristicas.append(oscar)
                else: 
                    lista_caracteristicas.append("Nominado")
            except: 
                lista_caracteristicas.append("No tiene galardones ni nominaciones")
                

            
            lista_caracteristicas.append(actor)  
            lista_caracteristicas.append(conocidas.split("\n")[::4])
            lista_caracteristicas.append(profesiones)
            lista_detalles_actores.append(tuple(lista_caracteristicas))
            driver.back()
        
    driver.close()
    
    return lista_detalles_actores  


In [35]:
caracteristicas_imdb_actores=caracteristicas_actores(lista_tupla_actores)

  0%|          | 0/10 [00:00<?, ?it/s]

Martin Aslan


 10%|█         | 1/10 [00:22<03:18, 22.00s/it]

Héctor Atienza


 20%|██        | 2/10 [00:40<02:40, 20.08s/it]

...

Jodi Shilling


 30%|███       | 3/10 [00:57<02:12, 18.99s/it]

Jane Alderman


 40%|████      | 4/10 [01:17<01:58, 19.71s/it]

William Zabka


In [49]:
caracteristicas_imdb_actores

[('1981',
  'No tiene galardones ni nominaciones',
  'Ali Liebert',
  [''],
  'Reparto, Dirección, Producción'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Troy Ruptash',
  [''],
  'Reparto, Producción, Dirección'),
 ('octubre',
  'No tiene galardones ni nominaciones',
  'Alex Feldman',
  [''],
  'Reparto, Dirección, Producción'),
 ('1980',
  'No tiene galardones ni nominaciones',
  'Ryan Northcott',
  [''],
  'Reparto, Dirección, Guion'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Pierce Briggs',
  [''],
  'Reparto'),
 ('Desconocido', 'Nominado', 'David Ley', [''], 'Reparto'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Natalie Yatsina',
  [''],
  'Reparto, Equipos adicionales, Arte'),
 ('Desconocido', 'Nominado', 'Daniel Mazepa', [''], 'Reparto'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Garrett Chomlak',
  [''],
  'Reparto'),
 ('Desconocido',
  'Nominado',
  'They Who Surround Us',
  ['Sin datos'],
  '2020, 1h 

In [50]:
# Una vez que tengamos la lista de las caracteristicas de los actores, tendremos que 
# transformar la lista de tuplas en lista de listas 
lista_limpieza = [list(x) for x in caracteristicas_imdb_actores]

In [67]:
# Eliminamos la lista de 'por qué es conocido' y lo transformamos a strings.
for indice,actriz in enumerate(lista_limpieza): 
    for elemento in actriz:
        if type(elemento)==list:
            elemento_limpio= ", ".join(elemento)
            lista_limpieza[indice][3] = elemento_limpio
            print(elemento_limpio)

In [52]:
# Convertimos nuevamente la lista de listas en lista de tuplas
lista_limpieza_tupla = [tuple(x) for x in lista_limpieza]

In [53]:
# Este resultado es el que guardamos en csv para la base de datos
lista_limpieza_tupla

[('1981',
  'No tiene galardones ni nominaciones',
  'Ali Liebert',
  '',
  'Reparto, Dirección, Producción'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Troy Ruptash',
  '',
  'Reparto, Producción, Dirección'),
 ('octubre',
  'No tiene galardones ni nominaciones',
  'Alex Feldman',
  '',
  'Reparto, Dirección, Producción'),
 ('1980',
  'No tiene galardones ni nominaciones',
  'Ryan Northcott',
  '',
  'Reparto, Dirección, Guion'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Pierce Briggs',
  '',
  'Reparto'),
 ('Desconocido', 'Nominado', 'David Ley', '', 'Reparto'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Natalie Yatsina',
  '',
  'Reparto, Equipos adicionales, Arte'),
 ('Desconocido', 'Nominado', 'Daniel Mazepa', '', 'Reparto'),
 ('Desconocido',
  'No tiene galardones ni nominaciones',
  'Garrett Chomlak',
  '',
  'Reparto'),
 ('Desconocido',
  'Nominado',
  'They Who Surround Us',
  'Sin datos',
  '2020, 1h 28min'),
 ('1980', '